In [ ]:
import yfinance as  yf
import backtesting
import pandas as pd

In [ ]:
base_df = yf.download('ZW=F', start='2022-01-01', end='2022-12-31')
base_df['Date'] = base_df.index
# view_low_volume_df = base_df[base_df['Volume'] < 1000]
# view_low_volume_df
base_df

In [ ]:
df = pd.read_csv('c_year_22.txt')
df['Report_Date_as_YYYY-MM-DD'] = pd.to_datetime(df['Report_Date_as_YYYY-MM-DD'])  # Convert the column to datetime if not already in datetime format

start_date = pd.to_datetime('2022-01-01')
end_date = pd.to_datetime('2022-12-31')

# Boolean indexing to select rows within the date range
selected_rows = df[(df['Report_Date_as_YYYY-MM-DD'] >= start_date) & (df['Report_Date_as_YYYY-MM-DD'] <= end_date) & (df['Market_and_Exchange_Names'] == 'WHEAT-SRW - CHICAGO BOARD OF TRADE')]
selected_rows.set_index('Report_Date_as_YYYY-MM-DD', inplace=True)
selected_rows

In [ ]:
merged_df = base_df.merge(selected_rows, left_on=base_df.index, right_on='Report_Date_as_YYYY-MM-DD', how = 'outer')
merged_df.columns
merged_df.set_index('Date', inplace=True)
merged_df
# backtest 需要的dataframe的index需要是datetime

In [92]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy, plot_heatmaps, crossover
from backtesting.test import SMA

    
class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 1
    n2 = 14
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        # precompute the open interest trend
        Open_interest_trend_flag = 0 # if the indecator changes exceed 10%, then we shall switch the position
        
        # Precompute the indecators

        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(self.sma1) > self.sma2).astype(int).diff().fillna(0)

        # signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        
        self.set_trailing_sl(8)
        self.set_atr_periods(50)

class ComplexStrategy(SmaCross):
    def init(self):
        super().init()
        # Precompute the indecators
        
        self.open_intersts_trend = pd.Series(self.data.Open_Interest_All).fillna(method = 'bfill')
        
        print(self.open_intersts_trend)
        
final_strategy = ComplexStrategy



In [93]:
final_strategy = ComplexStrategy
bt = Backtest(merged_df, final_strategy, cash=1e8, hedging=True,exclusive_orders=True, trade_on_close=True)
print(bt.run())

0      439485.0
1      439485.0
2      455595.0
3      455595.0
4      455595.0
         ...   
246    387675.0
247    387675.0
248         NaN
249         NaN
250         NaN
Length: 251, dtype: float64
Start                     2022-01-03 00:00:00
End                       2022-12-30 00:00:00
Duration                    361 days 00:00:00
Exposure Time [%]                   92.031873
Equity Final [$]                 118586520.68
Equity Peak [$]                   163922769.0
Return [%]                          18.586521
Buy & Hold Return [%]                4.485488
Return (Ann.) [%]                   18.667089
Volatility (Ann.) [%]               59.800363
Sharpe Ratio                         0.312157
Sortino Ratio                        0.617237
Calmar Ratio                         0.492999
Max. Drawdown [%]                  -37.864319
Avg. Drawdown [%]                  -14.168848
Max. Drawdown Duration      298 days 00:00:00
Avg. Drawdown Duration       81 days 00:00:00
# Trades      

In [ ]:
final_strategy = ComplexStrategy
bt = Backtest(merged_df, final_strategy, cash=1e8, hedging=True,exclusive_orders=True, trade_on_close=True)
results, heatmap = bt.optimize(n1=range(5, 20, 1), n2 = range(5, 20, 1), maximize='Sharpe Ratio', constraint=lambda p: p.n1 < p.n2, return_heatmap=True)
bt.plot(results = results)
plot_heatmaps(heatmap)
print(results)